<a href="https://colab.research.google.com/github/poip-boop/Patent_phrase_to_phrase_matching/blob/main/Patent_phrase_to_phrase_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries for project
import os
import pandas as pd
from datasets import Dataset,DatasetDict
from transformers import AutoTokenizer,AutoModelForSequenceClassification,TrainingArguments,Trainer
from scipy.stats import pearsonr
import numpy as np
import datasets

In [ ]:
creds = {"username":"enockkatui","key":"c892fd650139363de522432f3faa95c8"}

In [ ]:
# Working with paths
from pathlib import Path
import json

iskaggle = os.environ.get('KAGGLE_KERNEL_RUN_TYPE', '')

cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(json.dumps(creds))
    cred_path.chmod(0o600)

In [ ]:
# Download and extract data from kaggle
path = Path('us-patent-phrase-to-phrase-matching')

if not iskaggle and not path.exists():
  import zipfile,kaggle
  kaggle.api.competition_download_cli(str(path))
  zipfile.ZipFile(f'{path}.zip').extractall(path)

100%|██████████| 682k/682k [00:00<00:00, 903MB/s]

In [ ]:
if iskaggle:
    path = Path('../input/us-patent-phrase-to-phrase-matching')
    ! pip install -q datasets

In [ ]:
# Check data available
!ls {path}

sample_submission.csv  test.csv  train.csv


In [ ]:
# Load data
df = pd.read_csv(path/'train.csv')

# Exploratory Data Analysis

In [ ]:
df

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
...,...,...,...,...,...
36468,8e1386cbefd7f245,wood article,wooden article,B44,1.00
36469,42d9e032d1cd3242,wood article,wooden box,B44,0.50
36470,208654ccb9e14fa3,wood article,wooden handle,B44,0.50
36471,756ec035e694722b,wood article,wooden material,B44,0.75


In [ ]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,8d135da0b55b8c88,component composite coating,composition,H01
freq,1,152,24,2186


In [ ]:
# Create a combined input string from context, target, and anchor columns
df['input']='TEXT1:' + df.context + ';TEXT2:' + df.target +';ANC1:' + df.anchor

In [ ]:
df.input.head()

,input
0,TEXT1:A47;TEXT2:abatement of pollution;ANC1:ab...
1,TEXT1:A47;TEXT2:act of abating;ANC1:abatement
2,TEXT1:A47;TEXT2:active catalyst;ANC1:abatement
3,TEXT1:A47;TEXT2:eliminating process;ANC1:abate...
4,TEXT1:A47;TEXT2:forest region;ANC1:abatement


In [ ]:
ds = Dataset.from_pandas(df)

In [ ]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [ ]:
model_nm = 'microsoft/deberta-v3-small'

In [ ]:
tokz = AutoTokenizer.from_pretrained(model_nm)

tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/convert_slow_tokenizer.py:564: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
# Example tokenization
tokz.tokenize("Hi, this is an amazing project on NLP.")

['▁Hi', ',', '▁this', '▁is', '▁an', '▁amazing', '▁project', '▁on', '▁NLP', '.']

In [ ]:
def tok_func(x):
  """Tokenizes the 'input' field of a dataset example.

  Args:
    x: A dictionary-like object representing a single example from a dataset,
       expected to contain an 'input' key.

  Returns:
    The tokenized output of the 'input' field using the pre-trained tokenizer.
  """
  return tokz(x['input'])

In [ ]:
# Parallel application on each example
tok_ds = ds.map(tok_func, batched = True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [ ]:
row = tok_ds[0]
row['input'],row['input_ids']

('TEXT1:A47;TEXT2:abatement of pollution;ANC1:abatement',
 [1,
  54453,
  435,
  294,
  558,
  5753,
  346,
  104917,
  445,
  294,
  16191,
  297,
  17019,
  265,
  6435,
  346,
  64097,
  435,
  294,
  16191,
  297,
  17019,
  2])

In [ ]:
# Rename score to labels for transformers
tok_ds = tok_ds.rename_columns({'score':'labels'})

# Test and Validation sets

In [ ]:
eval_df = pd.read_csv(path/'test.csv')


In [ ]:
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,hybrid bearing,inorganic photoconductor drum,G02
freq,1,2,1,3


In [ ]:
dds = tok_ds.train_test_split(0.25, seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [ ]:
eval_df['input'] = 'TEXT1: ' + eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df).map(tok_func, batched=True)

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

In [ ]:
from scipy.stats import pearsonr
import numpy as np

# Dict for metrics
def corr_d(eval_pred):
  return {'pearson':pearsonr(*eval_pred)[0]}

# Training

In [ ]:
# Batch size and epoch allocation
bs = 128
epochs = 4

In [ ]:
# Learning rate
lr = 8e-5

In [ ]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    eval_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],
                  tokenizer=tokz, compute_metrics=corr_d)

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/tmp/ipython-input-3597993663.py:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(model, args, train_dataset=dds['train'], eval_dataset=dds['test'],


In [ ]:
trainer.train();

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 2, 'bos_token_id': 1}.


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.031458,0.776383
2,No log,0.025250,0.808559
3,0.035300,0.023164,0.817143
4,0.035300,0.024488,0.818663


In [ ]:
# Make predictions
preds = trainer.predict(eval_ds).predictions.astype(float)
preds

array([[ 0.54589844],
       [ 0.70166016],
       [ 0.50292969],
       [ 0.3737793 ],
       [ 0.03366089],
       [ 0.51269531],
       [ 0.49487305],
       [ 0.0287323 ],
       [ 0.31787109],
       [ 1.11328125],
       [ 0.20153809],
       [ 0.2734375 ],
       [ 0.75048828],
       [ 0.8203125 ],
       [ 0.74707031],
       [ 0.41186523],
       [ 0.27905273],
       [-0.0302124 ],
       [ 0.58544922],
       [ 0.30175781],
       [ 0.43505859],
       [ 0.27539062],
       [ 0.02668762],
       [ 0.2232666 ],
       [ 0.49853516],
       [-0.01676941],
       [-0.02888489],
       [-0.02154541],
       [-0.01853943],
       [ 0.69580078],
       [ 0.27783203],
       [ 0.00213432],
       [ 0.71240234],
       [ 0.50244141],
       [ 0.40429688],
       [ 0.19702148]])

In [ ]:
# Remove out-of-bounds-predictions
preds = np.clip(preds, 0, 1)

In [ ]:
# Submission file
submission = datasets.Dataset.from_dict({
    'id': eval_ds['id'],
    'score': preds
})

submission.to_csv('submission.csv', index=False)

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

1036